opening the txt file

In [2]:
with open("the-verdict.txt","r",encoding='utf-8') as f:
    raw_text = f.read()


simple tokenizer version 1.0

In [4]:
import re

class SimpleTokenizerv1: 

    """a class that converts RAW text into vocabulary (a list of sorted 
    unique words symbols in a corpus) and then makes tokens and assigns 
    token ids, it can also do the other way around"""

    def __init__ (self, raw_text):
        self.vocab = SimpleTokenizerv1.make_vocab(raw_text)
        self.str_to_int = self.vocab
        self.int_to_str = {i:s for s,i in self.vocab.items()}

    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text
    
    def make_vocab(raw_text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        all_words = sorted(set(preprocessed))

        vocab = {token: integer for integer, token in enumerate(all_words)}
        return vocab


text = 'this is the new tokenizer'

tokenizer = SimpleTokenizerv1(text)

ids = tokenizer.encode(text)
print(ids)

textt = tokenizer.decode(ids=ids)
print(textt)

[3, 0, 2, 1, 4]
this is the new tokenizer


### this tokenizer has support for unknown word that are not in vocab

In [6]:
class SimpleTokenizerv2: 

    """a class that converts RAW text into vocabulary (a list of sorted 
    unique words symbols in a corpus) and then makes tokens and assigns 
    token ids, it can also do the other way around"""

    def __init__ (self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed] #if the word is not in vocaab then |unk| that shiii
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

def make_vocab(raw_text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    all_words = sorted(set(preprocessed))
    all_words.extend(["<|endoftext|>", "<|unk|>"])
    vocab = {token: integer for integer, token in enumerate(all_words)}
    return vocab



In [7]:

vocabu = make_vocab(raw_text)
vocabu

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

In [8]:
tokenizer = SimpleTokenizerv2(vocabu)

text1 = 'this is the new tokenizer'
text2 = "hey is tokenizer v2"

text = " <|endoftext|> ".join((text1,text2))

ids=tokenizer.encode(text)
print(ids)
tokenizer.decode(ids)

[999, 584, 988, 1131, 1131, 1130, 1131, 584, 1131, 1131]


'this is the <|unk|> <|unk|> <|endoftext|> <|unk|> is <|unk|> <|unk|>'

#### byte-pair encoding wala tokenizer

In [10]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [12]:
ids = tokenizer.encode(raw_text)

In [13]:
len(ids)

5145

### starting off with making a dataset 

In [15]:
context = 4
print('input ----> target')
for i in range(1,context+1):
    context = ids[:i]
    desired = ids[i]
    print(tokenizer.decode(context), '---->',tokenizer.decode([desired])) #because desired is an int and decode acceptd a list

input ----> target
I ---->  H
I H ----> AD
I HAD ---->  always
I HAD always ---->  thought


## Now we need to create an efficient dataloader; that iterates over the input dataset and returns the inputs and targets as Pytorch tensors. 

In [17]:
import torch
from torch.utils.data import Dataset, DataLoader

In [18]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride): #uses sliding window to chunk the book into overlapping sequences of max_length (length of one chunk)
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i+1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
    def __len__(self): #returns total number of rows in the dataset
        return len(self.input_ids)

    def __getitem__(self, idx): #returns a single row from the dataset
        return self.input_ids[idx], self.target_ids[idx]
        

In [19]:
lodr = GPTDatasetV1(raw_text, tokenizer, 10, 1)

In [20]:
inputs = lodr.input_ids
targets = lodr.target_ids

## 2.6 A data loader to generate batches with input-with pairs

In [22]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, 
                         drop_last=True,num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle= shuffle,
        drop_last= drop_last,
        num_workers = num_workers
    )

    return dataloader 
    

In [23]:
dataloader= create_dataloader_v1(raw_text,batch_size=1,max_length=12,stride=2,shuffle=False)

#### the max_length is the lenght of each tensor. stride is how much jump we take from one batch to another, batch_size is the number of inpu-target pairs in one batch

In [25]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,
           257,  7026]]), tensor([[  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,
          7026, 15632]])]


In [26]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026,
         15632,   438]]), tensor([[ 1464,  1807,  3619,   402,   271, 10899,  2138,   257,  7026, 15632,
           438,  2016]])]


In [31]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


# 2.6 Encoding word positions

In [35]:
max_length = 4
dataloader = create_dataloader_v1(raw_text, batch_size = 8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print('Token IDs:\n',inputs)
print('\nInput Shape: \n', inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Input Shape: 
 torch.Size([8, 4])


In [39]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [57]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings)
                                    

tensor([[ 0.6013,  0.9191, -0.8033,  ..., -0.2072, -0.8240, -0.4563],
        [ 1.9586, -2.1272, -0.9310,  ...,  2.0047,  0.1878, -2.0400],
        [ 0.0327,  1.1478,  0.1594,  ...,  2.0197,  0.5266, -0.4836],
        [ 0.4185,  2.1593, -0.3320,  ..., -1.8505, -0.8514,  0.6207]],
       grad_fn=<EmbeddingBackward0>)


In [59]:
input_embeddings = token_embeddings + pos_embeddings

now we have added the positional embeddings to the actual token embeddings, this we will use to train the LLM|

# End of chapter 2